In [3]:
%load_ext autoreload
%autoreload 2
import feature_extract
import sys
sys.path.append("..")
import torch
import psql_methods as psql
import pickle
import alchemy_methods as alc
from tqdm import tqdm
import numpy as np
import image_utils as imgs
import opensea_methods as opse
import multiprocessing
import pandas as pd
import feature_utils as feat
import matplotlib.pyplot as plt
import counterfeit_utils as cfu
import cf_value as cfv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This notebook retreives the sales data for the top 1500 dataset that are used as controls in the SCM analysis

In [4]:
command = 'select slug,count(*) from slug_to_der group by slug'
data = psql.execute_commands([command])
df = pd.DataFrame(data,columns = ['slug','num_der'])
command = 'select slug,num from slug_to_counterfeit_num'
data = psql.execute_commands([command])
df_2 = pd.DataFrame(data,columns = ['slug','num_cf'])
merge_df = pd.merge(df,df_2,on='slug')
merge_df['diff']= merge_df['num_cf']-merge_df['num_der']
merge_df

,slug,num_der,num_cf,diff
0,azuki,11,50,39
1,hashmasks,1,1,0
2,mfers,1,20,19
3,fluf,10,20,10
4,degods,1,10,9
5,cool-cats-nft,9,20,11
6,boredapeyachtclub,6,120,114
7,doodles-official,2,50,48
8,karafuru,3,30,27
9,cryptopunks,1,150,149


In [5]:
import pickle
with open('graph_images_dataframe.pkl','rb') as f:
    features = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'graph_images_dataframe.pkl'

In [6]:
import pandas as pd

# Assuming df is your DataFrame
# Replace "slug" and "average_features" with your actual column names
columns_to_keep = ["Collection", "AverageFeatureVector"]

# Convert arrays to tuples in the "average_features" column
features["AverageFeatureVector"] = features["AverageFeatureVector"].apply(tuple)

# Create a new DataFrame with only distinct values
unique_df = features[columns_to_keep].drop_duplicates()

# Convert back the tuples to arrays if needed
unique_df["AverageFeatureVector"] = unique_df["AverageFeatureVector"].apply(list)
unique_df

NameError: name 'features' is not defined

In [12]:
del features

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from more_itertools import chunked
from psql_methods import execute_commands,batch_insert
from psycopg2 import sql
commands = []
data = []
for _, row in unique_df.iterrows():
    update_query = """
        UPDATE collection_table
        SET centroid = %s
        WHERE slug = %s
    """
    data.append((row["AverageFeatureVector"],row["Collection"]))
    commands.append(update_query)
execute_commands(commands,data,no_return=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
command = "SELECT * from collection_total_stats"
temp_stats_tuples = psql.execute_commands([command])
columns = ['volume','sales', 'avg_price','num_owners','market_cap','floor','symbol','slug']
stats_df = pd.DataFrame(temp_stats_tuples,columns=columns)

In [8]:
stats_df.sort_values(by='volume',ascending=False)

,volume,sales,avg_price,num_owners,market_cap,floor,symbol,slug
610,1.460340e+06,45416.0,32.154739,5422.0,252052.905422,25.499900,ETH,boredapeyachtclub
315,1.102118e+06,104449.0,10.551736,11158.0,86892.048380,4.380000,ETH,mutant-ape-yacht-club
319,7.225442e+05,72557.0,9.958298,4136.0,71987.081373,6.980000,ETH,azuki
554,6.161550e+05,182701.0,3.372477,15460.0,27649.297609,0.479899,ETH,otherdeed
918,5.253117e+05,261535.0,2.008571,42136.0,159523.136944,1.000000,ETH,art-blocks
...,...,...,...,...,...,...,...,...
10239,0.000000e+00,0.0,0.000000,238.0,0.000000,0.009000,ETH,caesium-by-none
11211,0.000000e+00,0.0,0.000000,193.0,0.000000,0.000000,,nakabaddies
11930,0.000000e+00,0.0,0.000000,211.0,0.000000,0.005500,ETH,based-degens-eth
8538,0.000000e+00,0.0,0.000000,1474.0,0.000000,0.004500,ETH,bustling-bees


In [9]:
command = "SELECT * from slug_to_type"
rows = psql.execute_commands([command])
columns = ['slug','type']
type_df = pd.DataFrame(rows,columns=columns)
type_and_stats = pd.merge(type_df,stats_df, on='slug')
type_and_stats_pfp = type_and_stats.query("type=='pfps' or type==''").sort_values(by='volume',ascending=False)

In [10]:
new_top_300 = type_and_stats_pfp.head(300)

In [29]:
command = "select distinct slug from cf_sales"
rows = psql.execute_commands([command])
done_slugs = [row[0] for row in rows]

In [30]:
matching_ids = new_top_300[new_top_300['slug'].isin(done_slugs)]
unmatched_ids = [id_val for id_val in done_slugs if id_val not in matching_ids['slug'].values]
# Get the count of matching IDs
matching_count = len(matching_ids)

In [31]:
matching_count

79

In [34]:
type_df.query(f"slug in {unmatched_ids}")

,slug,type
280,adidasoriginals,memberships
1334,bored-ape-chemistry-club,art
1850,chromie-squiggle-by-snowfro,art
1915,clonex-mintvial,art
1984,collectvoxtownstar,gaming
2134,creatureworld,art
2210,cryptoadz-by-gremplin,art
2302,cryptopunks,pfps
2351,curiocardswrapper,art
2392,cyberkongz-babies,gaming


In [33]:
type_df.query('slug=="cryptoadz-by-gremplin"')

,slug,type
2210,cryptoadz-by-gremplin,art


In [ ]:
slugs = list(new_top_300['slug'].values)
cfv.retreive_full_time_series(slugs)

('moshimochi', '2183', '0x3f4db38b5dac8288e1ec48cf30328edd434d22fb', '0x54bc0cf4c57f70602909259a3eb2b267cb758925', 1647135781, 0.078, 'ETH', '0xe3dba2fa69413de895c65a231b0431a74e91573185ef7866d85952ee61fbdde2')
2022-03-13 01:43:01
('moshimochi', '7875', '0x4f76bd26e62f710660ec14becfb5dfbd72ef262e', '0x97b55c22f399f7f48ca3b672193178ec10433ab4', 1644065522, 0.15000000000000002, 'ETH', '0x5e674cc2054f3e268868901aee383a81be8ce19b34abbdd477f0d9b80c34a759')
2022-02-05 12:52:02
('moshimochi', '6422', '0x8a7905156a57a4f591666e33100ccc4db9ac110b', '0x67e32e39bfcb2f793d43fa3b09c6a90d08c7f94f', 1643737045, 0.21800000000000003, 'ETH', '0x42f81f4eacd3eb24013d9ad6d53a50197b691c4d34b53fec277fa6ecb577a1fb')
2022-02-01 17:37:25
('moshimochi', '6798', '0x52ff35c1e5af02a1c7370f24b50b312e66e1bf90', '0xdefbffe64156f9b5e854206a62796e08df230948', 1643640043, 0.35000000000000003, 'ETH', '0xf1fcf4d44da37e9aad8f2b6656bc3fc17ee3d6d499b667b35bb42c5f9cfcf9e6')
2022-01-31 14:40:43
('moshimochi', '7235', '0xd0471582

In [11]:
new_top_300 = type_and_stats_pfp.head(1500)

In [12]:
new_top_300

,slug,type,volume,sales,avg_price,num_owners,market_cap,floor,symbol
1261,boredapeyachtclub,pfps,1.460340e+06,45416.0,32.154739,5422.0,252052.905422,25.499900,ETH
5850,mutant-ape-yacht-club,pfps,1.102118e+06,104449.0,10.551736,11158.0,86892.048380,4.380000,ETH
747,azuki,pfps,7.225442e+05,72557.0,9.958298,4136.0,71987.081373,6.980000,ETH
1755,clonex,pfps,4.153179e+05,66381.0,6.256578,9619.0,25277.749247,1.219999,ETH
7056,proof-moonbirds,pfps,3.447647e+05,49659.0,6.942642,5806.0,10879.602797,1.195000,ETH
...,...,...,...,...,...,...,...,...,...
7475,robomito,pfps,1.943656e+02,4033.0,0.048194,941.0,0.000000,0.009000,ETH
7705,scremlins,pfps,1.937249e+02,7080.0,0.027362,954.0,0.000000,0.002000,ETH
3775,gnomeville,pfps,1.934596e+02,7021.0,0.027554,3654.0,0.000000,0.007500,ETH
8259,steedz-of-degen,pfps,1.932696e+02,1343.0,0.143909,1227.0,0.000000,0.012500,ETH


In [14]:
slugs = ['clonex']
cfv.retreive_full_time_series(slugs)

('clonex', '8790', '0xc970d150e79dfe672332d4af0902ef23955a189b', '0xc8700474a7f870509ac6820e05be1d457c15808b', 1712531435, 0.6123000000000001, 'WETH', '0xf03956f4a19344889ebc25ce55631acf99abc202eb419fa3eb17b12befd99d93')
2024-04-07 23:10:35
('clonex', '17310', '0x146a04f39c8944ca7336d285761f003e574e9426', '0x4819c54d3dd9b2af9b6535e518c85b6430160528', 1710024599, 0.7200000000000001, 'WETH', '0x069585950db59f0ab0e3b805f62de5f3a2351ac10d710fbd31b89c937c5121d4')
2024-03-09 22:49:59
('clonex', '9231', '0x29469395eaf6f95920e59f858042f0e28d98a20b', '0x97dde01d61c697ff8d5f002ade8f055021ad946f', 1707665591, 1.25, 'WETH', '0xc33a31f71759421b197d686827b40322955cf61b10784862a6b1697edd37a32a')
2024-02-11 15:33:11
('clonex', '13768', '0xc821d3d24c27c713a9e2758086bd29b23f7381e2', '0x7f79c84a51da7b07450b9184c904901613d262ab', 1705074143, 1.0, 'WETH', '0x3775c91b407ac98c7f0b9028f202a511eb764260155736ac961391e302a4fb09')
2024-01-12 15:42:23
('clonex', '12277', '0x5d01bab3eae092789a3e70c5c9649b0b53d8282c